In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

In [2]:
dataset_path = os.listdir('dataset/train')

label_types = os.listdir('dataset/train')
print (label_types)  

['before', 'computer', 'drink']


In [3]:
rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('dataset/train' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('dataset/train' + '/' +item) + '/' + room))
    
# Build a dataframe        
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())

      tag                      video_name
0  before  dataset/train/before/05729.mp4
1  before  dataset/train/before/05730.mp4
2  before  dataset/train/before/05731.mp4
3  before  dataset/train/before/05732.mp4
4  before  dataset/train/before/05733.mp4
      tag                     video_name
35  drink  dataset/train/drink/17733.mp4
36  drink  dataset/train/drink/17734.mp4
37  drink  dataset/train/drink/65539.mp4
38  drink  dataset/train/drink/65540.mp4
39  drink  dataset/train/drink/69302.mp4


In [4]:
df = train_df.loc[:,['video_name','tag']]
df
df.to_csv('train.csv')


In [5]:
dataset_path = os.listdir('dataset/test')
print(dataset_path)

room_types = os.listdir('dataset/test')
print("Types of activities found: ", len(dataset_path))

rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('dataset/test' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('dataset/test' + '/' +item) + '/' + room))
    
# Build a dataframe        
test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','tag']]
df
df.to_csv('test.csv')

['before', 'computer', 'drink']
Types of activities found:  3
        tag                       video_name
0    before    dataset/test/before/05727.mp4
1    before    dataset/test/before/05728.mp4
2  computer  dataset/test/computer/12311.mp4
3  computer  dataset/test/computer/12312.mp4
4     drink     dataset/test/drink/17710.mp4
        tag                       video_name
0    before    dataset/test/before/05727.mp4
1    before    dataset/test/before/05728.mp4
2  computer  dataset/test/computer/12311.mp4
3  computer  dataset/test/computer/12312.mp4
4     drink     dataset/test/drink/17710.mp4


In [6]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.sample(10)

Total videos for training: 40
Total videos for testing: 5


,Unnamed: 0,video_name,tag
16,16,dataset/train/computer/12315.mp4,computer
24,24,dataset/train/computer/12328.mp4,computer
14,14,dataset/train/computer/12313.mp4,computer
3,3,dataset/train/before/05732.mp4,before
33,33,dataset/train/drink/17723.mp4,drink
13,13,dataset/train/before/65167.mp4,before
1,1,dataset/train/before/05730.mp4,before
21,21,dataset/train/computer/12320.mp4,computer
10,10,dataset/train/before/05743.mp4,before
23,23,dataset/train/computer/12327.mp4,computer


In [7]:
#from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [8]:
# The following two methods are taken from this tutorial:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [9]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

In [14]:
#label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
#print(label_processor.get_vocabulary())

labels = train_df["tag"].values
#labels = label_processor(labels[..., None]).numpy()
labels


array(['before', 'before', 'before', 'before', 'before', 'before',
       'before', 'before', 'before', 'before', 'before', 'before',
       'before', 'before', 'computer', 'computer', 'computer', 'computer',
       'computer', 'computer', 'computer', 'computer', 'computer',
       'computer', 'computer', 'computer', 'drink', 'drink', 'drink',
       'drink', 'drink', 'drink', 'drink', 'drink', 'drink', 'drink',
       'drink', 'drink', 'drink', 'drink'], dtype=object)

In [30]:
labels = list(np.unique(train_df["tag"]))
print(labels)

['before', 'computer', 'drink']


In [32]:
def label_to_tensor(st):
    f_list = list(np.unique(train_df["tag"]))
    f_list.sort()
    st_list = []
    for s in st:
            s_index = f_list.index(s)
            st_list.append(s_index)
    return tf.convert_to_tensor(st_list, dtype=tf.float32)
labels = label_to_tensor(labels).numpy()
labels


array([0., 1., 2.], dtype=float32)

In [33]:
#Define hyperparameters

IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [34]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    
    ##take all classlabels from train_df column named 'tag' and store in labels
    labels = df["tag"].values
    
    #convert classlabels to label encoding
    labels = label_to_tensor(labels).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels

train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")



Frame features in train set: (40, 20, 2048)
Frame masks in train set: (40, 20)
train_labels in train set: (40,)
test_labels in train set: (5,)


In [37]:

# Utility for our sequence model.
def get_sequence_model():
    class_vocab = 3

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 30
# Utility for running experiments.
def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model

_, sequence_model = run_experiment()

NotImplementedError: Cannot convert a symbolic Tensor (gru/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported